<a href="https://colab.research.google.com/github/quotation3/project/blob/master/SNS%ED%94%84%EB%A1%9C%ED%8C%8C%EC%9D%BC%EB%A7%81/%ED%85%8D%EC%8A%A4%ED%8A%B8%EA%B8%B0%EB%B0%98svm%EB%AA%A8%EB%8D%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install soynlp

     |████████████████████████████████| 419kB 8.0MB/s 


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# train_df=pd.read_csv("train_data.csv",engine="python",encoding="utf-8")[:1000]
# test_df=pd.read_csv("test_data.csv",engine="python",encoding="utf-8")[:1000]
# df=pd.concat([train_df,test_df])
df=pd.read_csv("/content/drive/My Drive/insta_profiling1/Data_Preprocessing/Data_Set/gender_post.csv",engine="python",encoding="utf-8")
df

,Unnamed: 0,insta_id,content,label
0,0,seo_preme,김밥집🍙🍲,0.0
1,1,seo_preme,이모 잘 입을게🙌 @d_nuum,0.0
2,2,seo_preme,짧은머리 3일차 아직도 어색 #부적응 #더움 #목은시원하네,0.0
3,3,seo_preme,생에 처음이자 마지막이 될 수도 있는 장발 생활을 정리함 #2년 #장발 #안녕,0.0
4,4,seo_preme,알타리 귀여움,0.0
...,...,...,...,...
198437,197079,ssenunninuskin,#감사하는 하루 #파마넥스 #내곡동맘 #서초구뉴스키너 #성장,1.0
198438,197080,ssenunninuskin,#서초구개맘 #내곡동맘 #개형 개엄마 #개귀염 #개심심 #개피곤,1.0
198439,197081,ssenunninuskin,#개시원 #바디바 #퓨어클렌징젤 #AP24 이만 닦아도 시원 물티슈바디바로 빨아서...,1.0
198440,197082,ssenunninuskin,#양고기 #가족 #아들운전면허합격 차키를 앞으로 숨겨야되나...,1.0


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
insta_id=df["insta_id"].tolist()
content=df["content"].tolist()
label=df["label"].tolist()

In [ ]:
df=pd.DataFrame({"insta_id":insta_id,"content":content,"label":label})

In [ ]:
def del_escape(text):
    char_ls=list(text)
    reduced_ls=[char for char in char_ls if "{c!r}".format(c=char).replace("'","")==char]
    return "".join(reduced_ls)

In [ ]:
data=df.copy()
data=data.dropna()
data["content"]=data["content"].apply(del_escape)
data=data.dropna()

In [ ]:
data

,insta_id,content,label
0,seo_preme,김밥집🍙🍲,0.0
1,seo_preme,이모 잘 입을게🙌 @d_nuum,0.0
2,seo_preme,짧은머리 3일차 아직도 어색 #부적응 #더움 #목은시원하네,0.0
3,seo_preme,생에 처음이자 마지막이 될 수도 있는 장발 생활을 정리함 #2년 #장발 #안녕,0.0
4,seo_preme,알타리 귀여움,0.0
...,...,...,...
198436,ssenunninuskin,#개엄마#내곡동맘 #에너지 채워야해 #오버드라이브 #서초구내곡동 #개스타그램 #...,1.0
198437,ssenunninuskin,#감사하는 하루 #파마넥스 #내곡동맘 #서초구뉴스키너 #성장,1.0
198438,ssenunninuskin,#서초구개맘 #내곡동맘 #개형 개엄마 #개귀염 #개심심 #개피곤,1.0
198439,ssenunninuskin,#개시원 #바디바 #퓨어클렌징젤 #AP24 이만 닦아도 시원 물티슈바디바로 빨아서...,1.0


In [ ]:
data["content"].to_csv("gender_post_content.csv",index=False)

In [ ]:
from soynlp import DoublespaceLineCorpus

corpus_path = "gender_post_content.csv"
sentences = DoublespaceLineCorpus(corpus_path, iter_sent=True)

In [ ]:
from soynlp.word import WordExtractor
from soynlp.noun import LRNounExtractor_v2
noun_extractor = LRNounExtractor_v2()

noun_extractor.train(sentences) # list of str or like
nouns = noun_extractor.extract(min_noun_frequency=10)

[Noun Extractor] use default predictors
[Noun Extractor] num features: pos=3929, neg=2321, common=107
[Noun Extractor] counting eojeols
[EojeolCounter] n eojeol = 1085719 from 350519 sents. mem=0.603 Gb                    
[Noun Extractor] complete eojeol counter -> lr graph
[Noun Extractor] has been trained. #eojeols=3999318, mem=3.619 Gb
[Noun Extractor] batch prediction was completed for 187340 words
[Noun Extractor] checked compounds. discovered 170441 compounds
[Noun Extractor] postprocessing detaching_features : 15055 -> 14838
[Noun Extractor] postprocessing ignore_features : 14838 -> 14589
[Noun Extractor] postprocessing ignore_NJ : 14589 -> 14297
[Noun Extractor] 14297 nouns (170441 compounds) with min frequency=10
[Noun Extractor] flushing was done. mem=4.519 Gb                    
[Noun Extractor] 50.45 % eojeols are covered


In [ ]:
user_seg_df=data.pivot_table(index="insta_id",values=["label"],aggfunc="median")
user_seg_df=user_seg_df.reset_index()
user_seg_df

,insta_id,label
0,00000_min04,0.0
1,00_89h,0.0
2,00_k.d.n,0.0
3,010_28888200,0.0
4,0129_98,0.0
...,...,...
2438,zosia_josia,1.0
2439,zxxyeon,0.0
2440,zzang.guns,0.0
2441,zzu_nny0508,0.0


In [ ]:
user_text_df=data.pivot_table(index="insta_id",values=["content"],aggfunc="sum")
user_text_df=user_text_df.reset_index()
user_text_df

,insta_id,content
0,00000_min04,배구에 다시 반해버렸다Gs 화이팅... 박혜민 선수 화이팅..❤❤Good frien...
1,00_89h,내가 자주 간식사주는 애들미안하다 이거 보여주려고 어그로 끌었다좋겠다 넌 소리소앞에...
2,00_k.d.n,🇰🇷해병대🇰🇷마지막.... #잘가라 #머리야기분 전환잘 부탁한다~! #행운이따르길언...
3,010_28888200,뿌리 뽑힌 나무를 다시 심는다고 시멘트까지 붓는데 다시 안쓰러지나.. 시멘트질이 영...
4,0129_98,#전주#선거 안하는(못하는x) 사람이랑 겸상안함저녁먹고가려고 9시차 예약했더니 8시...
...,...,...
2438,zosia_josia,M◇E◇M◇O◇R◇I◇E◇S __________________________ Es ...
2439,zxxyeon,아보카도라떼 10년전 미얀마에 갔을때 적은 돈이지만 사치를 부려보자며 갔던 큰 레...
2440,zzang.guns,#고기##삼겹살#소고기#복분자#좋은데이#소주#부산#기장#캠핑#노지#파도소리에#취해#...
2441,zzu_nny0508,저는 소중한 벗 영화(@imglory__ )의 지목으로 코로나19극복을위한 시편필사...


In [ ]:
dic={}

for term in nouns:
    try:
        reduced_term=term.replace("(","").replace(")","")
        dic[reduced_term]=[sum(data.loc[data["label"]==0]["content"].str.contains(reduced_term)),sum(data["label"]==0)-sum(data.loc[data["label"]==0]["content"].str.contains(reduced_term)),
           sum(data.loc[data["label"]!=0]["content"].str.contains(reduced_term)),sum(data["label"]!=0)-sum(data.loc[data["label"]!=0]["content"].str.contains(reduced_term))]
    except:
        print("not ordinary")

dic  

not ordinary
not ordinary
not ordinary
not ordinary
not ordinary
not ordinary
not ordinary
not ordinary
not ordinary


{'리얼이스케이프첼린지': [0, 67493, 196, 125030],
 '서초씨앤씨·애니스타': [0, 67493, 49, 125177],
 '감염증코로나19': [0, 67493, 0, 125226],
 '#화이트365숙대점': [0, 67493, 53, 125173],
 '숙명한국어교육봉사단': [0, 67493, 18, 125208],
 'Conference': [3, 67490, 10, 125216],
 '스냅타임': [0, 67493, 68, 125158],
 '#삼육대아트앤디자인': [0, 67493, 16, 125210],
 '#커뮤니케이션디자인': [0, 67493, 36, 125190],
 '인스타그램/페이스북': [0, 67493, 8, 125218],
 'restaurant': [19, 67474, 261, 124965],
 'OKR스터디플래너': [0, 67493, 11, 125215],
 '서초씨앤씨애니스타': [0, 67493, 55, 125171],
 '총장선거관리위원회': [0, 67493, 3, 125223],
 '동탄클로버미술학원': [0, 67493, 352, 124874],
 '입니다AIESEC': [0, 67493, 12, 125214],
 '서초씨앤씨미술학원': [0, 67493, 67, 125159],
 '서울과학기술대학교': [5, 67488, 15, 125211],
 '커뮤니케이션디자인': [0, 67493, 46, 125180],
 '#선팔#맞팔#인친': [125, 67368, 93, 125133],
 '#공부스타그램맞팔': [20, 67473, 1, 125225],
 '#아이스아메리카노': [60, 67433, 128, 125098],
 '앙트러프러너십전공': [0, 67493, 15, 125211],
 'Education': [24, 67469, 30, 125196],
 'Christmas': [28, 67465, 66, 125160],
 '칼라PT필라테스': [0, 67493, 157, 12506

In [ ]:
mll_dm={}
mll_im={}
aic_dm={}
aic_im={}
relevance={}
eps=1e-12
for x in dic:
    mll_dm[x]=dic[x][0]*np.log(dic[x][0]+eps)+dic[x][1]*np.log(dic[x][1]+eps)+dic[x][2]*np.log(dic[x][2]+eps)+dic[x][3]*np.log(dic[x][3]+eps)-(len(data))*np.log(len(data))
for x in dic:
    mll_im[x]=sum(data["label"]==0)*np.log(sum(data["label"]==0))+sum(data["label"]!=0)*np.log(sum(data["label"]!=0))+(dic[x][0]+dic[x][2])*np.log(dic[x][0]+dic[x][2]+eps)+(dic[x][1]+dic[x][3])*np.log(dic[x][1]+dic[x][3]+eps)-2*(len(data))*np.log(len(data))    
for x in dic:
    aic_dm[x]=-2*mll_dm[x]+6
for x in dic:
    aic_im[x]=-2*mll_im[x]+4
for x in dic:
    if dic[x][0]/(dic[x][0]+dic[x][2]+eps)>dic[x][1]/(dic[x][1]+dic[x][3]+eps):
        relevance[x]=aic_im[x]-aic_dm[x]+2
    else:
        relevance[x]=aic_dm[x]-aic_im[x]-2

In [ ]:
import pickle

# save data
# with open('dic.pickle','wb') as dic_label0:
#     pickle.dump(dic), dic_label0)


file=open('dic_label0','wb')
pickle.dump(dic,file)
file.close()

In [ ]:
characteristic_term={}
i=0
for x in relevance:
    if relevance[x]>0:
        characteristic_term[i]=x
        i+=1
characteristic_term

{0: '#선팔#맞팔#인친',
 1: '#공부스타그램맞팔',
 2: 'Education',
 3: '#카라멜마끼아또',
 4: '#크리스마스트리',
 5: '#비수_결국_베',
 6: '#등심스테이크',
 7: 'Amerigo',
 8: 'YouTube',
 9: '#패러글라이딩',
 10: '#믹스견은사랑',
 11: '#선팔하면맞팔',
 12: '학생부종합전형',
 13: '#에어소프트게',
 14: '#하고싶은거다',
 15: '#우정스타그램',
 16: '#03년생공스',
 17: '#별마당도서관',
 18: '축하해주신분들',
 19: '#인테리어디자',
 20: '크리스마스🎄',
 21: '사랑제일교회',
 22: '#아메리카노',
 23: '애니멀특전대',
 24: '#2017년',
 25: '"코로나19',
 26: '#코로나19',
 27: '게스트하우스',
 28: '코타키나발루',
 29: '#말레이시아',
 30: '#오스트리아',
 31: '#2018년',
 32: '#오늘도수고',
 33: '#스크린골프',
 34: '#만화캐릭터',
 35: '블라디보스톡',
 36: '#2020년',
 37: '#마지막사진',
 38: '#중학교친구',
 39: '#군인아저씨',
 40: 'HEROES',
 41: 'School',
 42: '#인스타그램',
 43: '좋은사람들과',
 44: 'school',
 45: '#캘리포니아',
 46: '카카오프렌즈',
 47: '#남는건사진',
 48: 'sunset',
 49: '감사합니다.',
 50: '금가루예술단',
 51: '카카오스토리',
 52: '#라스베가스',
 53: '뜨겁개핫도그',
 54: '인스타그램',
 55: '#좋은사람들',
 56: '#너의이름은',
 57: '#아쿠아리움',
 58: '#석고방향제',
 59: '#로제파스타',
 60: '#달고나라떼',
 61: '10시30분',
 62: '#코로나극복',
 63: '발렌타인데이',
 64: '#화이트비치',
 65: '#

In [ ]:
X=np.array(user_text_df.loc[user_text_df["insta_id"]!=target_user,"content"].apply(lambda x:[int(characteristic_term[i] in x) for i in range(len(characteristic_term))]).tolist())
y=np.array((user_seg_df.loc[user_text_df["insta_id"]!=target_user,"label"]==0).apply(int))

# X_train=np.array(train_user_text_df["content"].apply(lambda x:[int(characteristic_term[i] in x) for i in range(len(characteristic_term))]).tolist())
# y_train=np.array((train_user_seg_df["label"]==0).apply(int))

In [ ]:
# train_df=pd.DataFrame({"insta_id":train_insta_id,"content":train_content,"label":train_label})
# test_df=pd.DataFrame({"insta_id":test_insta_id,"content":test_content,"label":test_label})

In [ ]:
# train_data=train_df.copy()
# train_data=train_data.dropna()
# train_data["content"]=train_data["content"].apply(del_escape)
# train_data=train_data.dropna()
# test_data=test_df.copy()
# test_data=test_data.dropna()
# test_data["content"]=test_data["content"].apply(del_escape)
# test_data=test_data.dropna()

In [ ]:
# train_user_seg_df=train_data.pivot_table(index="insta_id",values=["label"],aggfunc="median")
# train_user_seg_df=train_user_seg_df.reset_index()
# test_user_seg_df=test_data.pivot_table(index="insta_id",values=["label"],aggfunc="median")
# test_user_seg_df=test_user_seg_df.reset_index()
# train_user_text_df=train_data.pivot_table(index="insta_id",values=["content"],aggfunc="sum")
# train_user_text_df=train_user_text_df.reset_index()
# test_user_text_df=test_data.pivot_table(index="insta_id",values=["content"],aggfunc="sum")
# test_user_text_df=test_user_text_df.reset_index()

In [ ]:
# X_train=np.array(train_user_text_df["content"].apply(lambda x:[int(characteristic_term[i] in x) for i in range(len(characteristic_term))]).tolist())
# y_train=np.array((train_user_seg_df["label"]==0).apply(int))
# X_test=np.array(test_user_text_df["content"].apply(lambda x:[int(characteristic_term[i] in x) for i in range(len(characteristic_term))]).tolist())
# y_test=np.array((test_user_seg_df["label"]==0).apply(int))

In [ ]:
from sklearn.svm import SVC
model=SVC(probability=True)
model.fit(X,y)


SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=True, random_state=None, shrinking=True, tol=0.001,
    verbose=False)

In [ ]:
X_target_user=np.array(user_text_df.loc[user_text_df["insta_id"]==target_user,"content"].apply(lambda x:[int(characteristic_term[i] in x) for i in range(len(characteristic_term))]).tolist())
# y_target_user=np.array((user_seg_df.loc[user_text_df["insta_id"]==target_user,"label"]==0).apply(int))

In [ ]:
pred_0=model.predict_proba(X_target_user)
pred_0

array([[0.36647299, 0.63352701]])

In [ ]:
pred_ls=[pred_0]
pred_ls

[array([[0.36647299, 0.63352701]])]

In [ ]:
# label=1 인경우 되풀이
dic={}

for term in nouns:
    try:
        reduced_term=term.replace("(","").replace(")","")
        dic[reduced_term]=[sum(data.loc[data["label"]==1]["content"].str.contains(reduced_term)),sum(data["label"]==1)-sum(data.loc[data["label"]==1]["content"].str.contains(reduced_term)),
           sum(data.loc[data["label"]!=1]["content"].str.contains(reduced_term)),sum(data["label"]!=1)-sum(data.loc[data["label"]!=1]["content"].str.contains(reduced_term))]
    except:
        print("not ordinary")

dic

not ordinary
not ordinary
not ordinary
not ordinary
not ordinary
not ordinary
not ordinary
not ordinary
not ordinary


{'감염증코로나19': [0, 125226, 0, 67493],
 '#화이트365숙대점': [53, 125173, 0, 67493],
 '인스타그램/페이스북': [8, 125218, 0, 67493],
 '리얼이스케이프첼린지': [196, 125030, 0, 67493],
 '서초씨앤씨·애니스타': [49, 125177, 0, 67493],
 'restaurant': [261, 124965, 19, 67474],
 '숙명한국어교육봉사단': [18, 125208, 0, 67493],
 'Conference': [10, 125216, 3, 67490],
 '스냅타임': [68, 125158, 0, 67493],
 '#삼육대아트앤디자인': [16, 125210, 0, 67493],
 '#커뮤니케이션디자인': [36, 125190, 0, 67493],
 '#선팔#맞팔#인친': [93, 125133, 125, 67368],
 '#공부스타그램맞팔': [1, 125225, 20, 67473],
 '동탄클로버미술학원': [352, 124874, 0, 67493],
 '앙트러프러너십전공': [15, 125211, 0, 67493],
 'Education': [30, 125196, 24, 67469],
 '총장선거관리위원회': [3, 125223, 0, 67493],
 '서초씨앤씨애니스타': [55, 125171, 0, 67493],
 '입니다AIESEC': [12, 125214, 0, 67493],
 '서초씨앤씨미술학원': [67, 125159, 0, 67493],
 'OKR스터디플래너': [11, 125215, 0, 67493],
 'Christmas': [66, 125160, 28, 67465],
 '서울과학기술대학교': [15, 125211, 5, 67488],
 '커뮤니케이션디자인': [46, 125180, 0, 67493],
 '#아이스아메리카노': [128, 125098, 60, 67433],
 '#공스타그램맞팔': [86, 125140, 16, 6747

In [ ]:
target_user="zzu_nny0508" #label=1.0

In [ ]:
import pickle
file=open('/content/drive/My Drive/insta_profiling1/Data_Preprocessing/Data_Set/dic_label_1','wb')
pickle.dump(dic,file)
file.close()

In [ ]:
import pickle
with open("/content/drive/My Drive/insta_profiling1/Data_Preprocessing/Data_Set/dic_label_1","rb") as fr:
    dic = pickle.load(fr)

In [ ]:
mll_dm={}
mll_im={}
aic_dm={}
aic_im={}
relevance={}
eps=1e-12
for x in dic:
    mll_dm[x]=dic[x][0]*np.log(dic[x][0]+eps)+dic[x][1]*np.log(dic[x][1]+eps)+dic[x][2]*np.log(dic[x][2]+eps)+dic[x][3]*np.log(dic[x][3]+eps)-(len(data))*np.log(len(data))
for x in dic:
    mll_im[x]=sum(data["label"]==1)*np.log(sum(data["label"]==1))+sum(data["label"]!=1)*np.log(sum(data["label"]!=1))+(dic[x][0]+dic[x][2])*np.log(dic[x][0]+dic[x][2]+eps)+(dic[x][1]+dic[x][3])*np.log(dic[x][1]+dic[x][3]+eps)-2*(len(data))*np.log(len(data))    
for x in dic:
    aic_dm[x]=-2*mll_dm[x]+6
for x in dic:
    aic_im[x]=-2*mll_im[x]+4
for x in dic:
    if dic[x][0]/(dic[x][0]+dic[x][2]+eps)>dic[x][1]/(dic[x][1]+dic[x][3]+eps):
        relevance[x]=aic_im[x]-aic_dm[x]+2
    else:
        relevance[x]=aic_dm[x]-aic_im[x]-2

In [ ]:
characteristic_term={}
i=0
for x in relevance:
    if relevance[x]>0:
        characteristic_term[i]=x
        i+=1

In [ ]:
X=np.array(user_text_df.loc[user_text_df["insta_id"]!=target_user,"content"].apply(lambda x:[int(characteristic_term[i] in x) for i in range(len(characteristic_term))]).tolist())
y=np.array((user_seg_df.loc[user_text_df["insta_id"]!=target_user,"label"]==1).apply(int))

In [ ]:
from sklearn.svm import SVC
model=SVC(probability=True)
model.fit(X,y)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=True, random_state=None, shrinking=True, tol=0.001,
    verbose=False)

In [ ]:
X_target_user=np.array(user_text_df.loc[user_text_df["insta_id"]==target_user,"content"].apply(lambda x:[int(characteristic_term[i] in x) for i in range(len(characteristic_term))]).tolist())
# y_target_user=np.array((user_seg_df.loc[user_text_df["insta_id"]==target_user,"label"]==k).apply(int))

In [ ]:
pred_0=model.predict_proba(X_target_user)
pred_0

ValueError: ignored

In [ ]:
pred_ls = [pred_0]
pred_ls

[array([[0.79988965, 0.20011035],
        [0.93937552, 0.06062448],
        [0.82344229, 0.17655771],
        ...,
        [0.96607944, 0.03392056],
        [0.9308195 , 0.0691805 ],
        [0.02065919, 0.97934081]])]